In [31]:
#BATHYMETRY
import csv
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from scipy.interpolate import griddata
from shapely.geometry import Point, Polygon, MultiPoint
from shapely.ops import unary_union
from scipy.spatial import ConvexHull
# Configure Chrome options for headless mode
chrome_options = Options()
chrome_options.add_argument('--headless')  # Run Chrome in headless mode
chrome_options.add_argument('--disable-gpu')  # Disable GPU acceleration
# Function to extract coordinates from KML file
def extract_coordinates_from_kml(kml_file):
    coordinates = []
    with open(kml_file, 'r') as file:
        kml_content = file.read()
        # Use regular expression to extract coordinates from KML content
        pattern = r'<coordinates>(.*?)</coordinates>'
        matches = re.findall(pattern, kml_content, re.DOTALL)
        for match in matches:
            # Split coordinates into individual points
            points = match.strip().split()
            for point in points:
                # Split latitude, longitude, and altitude values
                lat, lon, alt = point.split(',')
                coordinates.append((float(lon), float(lat)))
    return coordinates
# Path to the KML file containing coordinates
kml_file_path = '120_4.kml'
# Extract coordinates from KML file
coordinates = extract_coordinates_from_kml(kml_file_path)
# Create a WebDriver instance with headless mode
driver = webdriver.Chrome(options=chrome_options)
driver.implicitly_wait(20)  # Implicit wait to wait for elements
wait = WebDriverWait(driver, 20)  # Initialize WebDriverWait
driver.get('https://www.advancedconverter.com/map-tools/find-altitude-by-coordinates')
# Open a CSV file for writing
with open('120.csv', 'w', newline='') as csvfile:
    fieldnames = ['Latitude', 'Longitude', 'Depth']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # Iterate over coordinates extracted from KML file
    for lat, lon in coordinates:
        # Execute JavaScript to set latitude and longitude values directly
        driver.execute_script(f'document.getElementById("lat").value = "{lat}";')
        driver.execute_script(f'document.getElementById("lng").value = "{lon}";')

        # Click the insert point button
        button = wait.until(EC.element_to_be_clickable((By.ID, 'insertPoint')))
        button.click()

        try:
            # Wait for the depth information to be visible
            elevation = wait.until(EC.visibility_of_element_located((By.ID, 'outputDiv')))
            elevation_text = elevation.find_element(By.TAG_NAME, 'strong').text

            # Extract the depth value from the text
            elevation_value = elevation_text.split()[0]

            # Write to CSV
            writer.writerow({'Latitude': lat, 'Longitude': lon, 'Depth': elevation_value})
        except TimeoutException:
            # If depth information is not available, skip this coordinate
            print(f"Depth data not available for lat: {lat}, lon: {lon}")
driver.quit()
print("Depth data has been saved to csv")

Depth data has been saved to csv


In [32]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from scipy.interpolate import griddata

# Load data
df0 = pd.read_csv("120.csv")
lon = df0["Longitude"]
lat = df0["Latitude"]
elev = df0["Depth"]

# Create grid for surface plot
grid_lon = np.linspace(lon.min(), lon.max(), 100)
grid_lat = np.linspace(lat.min(), lat.max(), 100)
grid_lon, grid_lat = np.meshgrid(grid_lon, grid_lat)
grid_elev = griddata((lon, lat), elev, (grid_lon, grid_lat), method='cubic')

# Create contour plot using Plotly
contour_fig = go.Figure(go.Contour(x=lon, y=lat, z=elev, colorscale='spectral'))
contour_fig.update_layout(title='Bathymetric Contour Map (Plotly)', xaxis_title='Longitude', yaxis_title='Latitude')
contour_fig.update_traces(colorbar=dict(title='Depth', x=1.15))

# Show contour plot
contour_fig.show()

# Constants for converting meters to degrees
# Assuming the area is near the equator for simplicity, adjust as necessary for your region
meters_per_degree_lat = 111320  # Approximate number of meters per degree of latitude
meters_per_degree_lon = 111320  # This varies based on latitude, use the same value for simplicity

# Define the size of the area in meters
area_size_m = 120
total_area = 120*120
# Convert area size to degrees
area_size_deg_lat = area_size_m / meters_per_degree_lat
area_size_deg_lon = area_size_m / meters_per_degree_lon

# Prompt user for the center of the area
center_lon = lon.mean()
center_lat = lat.mean()

# Calculate the boundaries of the area
lon_min = center_lon - area_size_deg_lon / 2
lon_max = center_lon + area_size_deg_lon / 2
lat_min = center_lat - area_size_deg_lat / 2
lat_max = center_lat + area_size_deg_lat / 2

# Filter the dataframe for the selected area
filtered_df = df0[(df0["Longitude"] >= lon_min) & (df0["Longitude"] <= lon_max) & 
                 (df0["Latitude"] >= lat_min) & (df0["Latitude"] <= lat_max)]

# Save the filtered dataframe to a new CSV file
filtered_df.to_csv("filtered_area.csv", index=False)

# Highlight the selected area on the original contour plot
highlight = go.Scatter(
    x=[lon_min, lon_max, lon_max, lon_min, lon_min],
    y=[lat_min, lat_min, lat_max, lat_max, lat_min],
    mode='lines',
    line=dict(color='black', width=2),
    name='Selected Area'
)

contour_fig.add_trace(highlight)
contour_fig.show()

# Create 3D surface plot using Plotly for the selected area
selected_grid_lon = np.linspace(lon_min, lon_max, 100)
selected_grid_lat = np.linspace(lat_min, lat_max, 100)
selected_grid_lon, selected_grid_lat = np.meshgrid(selected_grid_lon, selected_grid_lat)
selected_grid_elev = griddata((lon, lat), elev, (selected_grid_lon, selected_grid_lat), method='cubic')

surface_fig = go.Figure(go.Surface(x=selected_grid_lon, y=selected_grid_lat, z=selected_grid_elev, colorscale='spectral'))
surface_fig.update_layout(title='Selected Area Bathymetric Surface Map (Plotly)', scene=dict(xaxis_title='Longitude', yaxis_title='Latitude', zaxis_title='Depth'))
surface_fig.update_traces(colorbar=dict(title='Depth', x=1.15))
surface_fig.show()


In [33]:
# DEFINING A THRESHOLD STATE
import numpy as np
import pandas as pd
from scipy.optimize import fsolve

def find_tl(l):
    f,k,s,t,d,pH,r=l
    f1=0.78*((s/35)**0.5)*(2.7183**(t/26))
    f2=42*(2.7183**(t/17))
    absorption_loss=((0.106*f1*f*f*(2.7183**((pH-8)/0.56)))/((f1*f1)+(f*f)))+(0.52*(1+t/43)*(s/35)*(f2*f*f*(2.7183**(-d/6)))/((f2*f2)+(f*f)))+(0.00049*f*f*(2.7183**(-(t/27+d/17))))
    transmission_loss=10*k*np.log10(r*1000)+absorption_loss*r
    return absorption_loss,transmission_loss

data=[14,2,35,4,-df0['Depth'].min()/1000,8,0.06]# THRESHOLD STATE [frequency, spreading factor ,salinity, temperature, depth, pH, propagation range]

alpha,tl_th=find_tl(data)
print("Threshold Transmission Loss:",tl_th)
print("Absorption Loss:",alpha)
# TESTING A SAMPLE STATE
def equation(r_new, tl_th):
    return 20 * np.log10(r_new) + 0.001*alpha * r_new - tl_th

def find_r_new(tl, initial_guess=1.0):
    r_new_solution = fsolve(equation, initial_guess, args=(tl,))
    return r_new_solution[0]
#print(find_r_new(34.081501776982066))

new_data=[14,2,35,4,0.215,8,0.06]
new_losses=find_tl(new_data)
print(f"New transmission loss: {new_losses[1]}")

if new_losses[1]>tl_th:
    loss=2*tl_th-new_losses[1]
else:
    loss=tl_th
print("New Propagation Range(in m):",find_r_new(loss))

Threshold Transmission Loss: 35.68863622563886
Absorption Loss: 2.0935202994331057
New transmission loss: 35.68716774338983
New Propagation Range(in m): 59.99999999999999


In [34]:
df=pd.read_csv('filtered_area.csv')
print(df)
df['Losses']=find_tl([14,2,35,4,-df['Depth']/1000,8,0.06])[1]
df['Range'] = df.apply(lambda row: find_r_new(2*tl_th-row['Losses'], alpha) if row['Losses'] > tl_th else 60, axis=1)
df = df.sort_values(by=['Latitude', 'Longitude']).reset_index(drop=True)
df

    Latitude  Longitude  Depth
0  18.444359  84.673036   -130
1  18.443478  84.674036   -133
2  18.444471  84.673561   -130
3  18.444161  84.673387   -131
4  18.443790  84.673429   -132
5  18.443699  84.673878   -133
6  18.444331  84.673989   -131
7  18.444181  84.673757   -131
8  18.444064  84.673771   -131


,Latitude,Longitude,Depth,Losses,Range
0,18.443478,84.674036,-133,35.688753,59.999205
1,18.443699,84.673878,-133,35.688753,59.999205
2,18.443790,84.673429,-132,35.688772,59.999073
3,18.444064,84.673771,-131,35.688792,59.998940
4,18.444161,84.673387,-131,35.688792,59.998940
5,18.444181,84.673757,-131,35.688792,59.998940
6,18.444331,84.673989,-131,35.688792,59.998940
7,18.444359,84.673036,-130,35.688811,59.998808
8,18.444471,84.673561,-130,35.688811,59.998808


In [35]:
selected_sensor = pd.DataFrame(columns=['Latitude', 'Longitude', 'Depth','Total Depth','Range'])

# Iterate over your data and add rows to the DataFrame
for i, j, depth, coverage_range in zip(df['Latitude'], df['Longitude'], df['Depth'], df['Range']):
    coverage_length = depth
    while coverage_length+coverage_range<0:
        coverage_length += coverage_range
        new_row = pd.DataFrame({'Latitude': [i], 'Longitude': [j], 'Depth': [coverage_length], 'Total Depth': depth,'Range': [coverage_range]})
        selected_sensor = pd.concat([selected_sensor, new_row], ignore_index=True)
selected_sensor = selected_sensor.sort_values(by=['Latitude', 'Longitude']).reset_index(drop=True)
selected_sensor

C:\Users\aksha\AppData\Local\Temp\ipykernel_10048\862810430.py:9: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



,Latitude,Longitude,Depth,Total Depth,Range
0,18.443478,84.674036,-73.000795,-133,59.999205
1,18.443478,84.674036,-13.001590,-133,59.999205
2,18.443699,84.673878,-73.000795,-133,59.999205
3,18.443699,84.673878,-13.001590,-133,59.999205
4,18.443790,84.673429,-72.000927,-132,59.999073
5,18.443790,84.673429,-12.001855,-132,59.999073
6,18.444064,84.673771,-71.001060,-131,59.998940
7,18.444064,84.673771,-11.002120,-131,59.998940
8,18.444161,84.673387,-71.001060,-131,59.998940
9,18.444161,84.673387,-11.002120,-131,59.998940


In [36]:
fig = go.Figure()

# Add surface map
fig.add_trace(go.Surface(
    x=df['Latitude'], y=df['Longitude'], z=df['Depth'],
    colorscale='turbo',
    opacity=1,
    showscale=False
))

# Add selected sensors as scatter plot
fig.add_trace(go.Scatter3d(
    x=selected_sensor['Latitude'], y=selected_sensor['Longitude'], z=selected_sensor['Depth'],
    mode='markers',
    marker=dict(
        size=4,
        color='red',
        symbol='circle'
    ),
    name='Selected Sensors'
))

# Customize layout
fig.update_layout(
    title='3D Surface Map with Selected Sensor Locations',
    scene=dict(
        xaxis_title='Latitude',
        yaxis_title='Longitude',
        zaxis_title='Depth',
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)

# Show the plot
fig.show()

In [23]:
#O(nlogn)
import pandas as pd
import numpy as np
import math
import heapq

# Function to calculate Euclidean distance considering latitude, longitude, and depth
def euclidean_distance_vectorized(lat1, lon1, depth1, lat2, lon2, depth2):
    x1 = lon1 * np.cos(np.radians(lat1))
    y1 = lat1
    z1 = depth1
    
    x2 = lon2 * np.cos(np.radians(lat2))
    y2 = lat2
    z2 = depth2

    horizontal_distance = np.sqrt((x2 - x1)**2 + (y2 - y1)**2) * 111320  # Convert degrees to meters
    vertical_distance = z2 - z1

    distance = np.sqrt(horizontal_distance**2 + vertical_distance**2)
    
    return distance

# Precompute all distances between sensors and grids
def precompute_distances(sensor_df, grid_df):
    lat1, lon1, depth1 = sensor_df['Latitude'].values, sensor_df['Longitude'].values, sensor_df['Depth'].values
    lat2, lon2, depth2 = grid_df['Latitude'].values, grid_df['Longitude'].values, grid_df['Depth'].values
    
    dist_matrix = np.zeros((lat1.size, lat2.size))
    
    for i in range(lat1.size):
        dist_matrix[i, :] = euclidean_distance_vectorized(lat1[i], lon1[i], depth1[i], lat2, lon2, depth2)
        
    return dist_matrix

def calculate_coverage(sensor_indices, dist_matrix, range_matrix):
    covered = np.any(dist_matrix[sensor_indices, :] <= range_matrix[sensor_indices, :], axis=0)
    return np.sum(covered) / dist_matrix.shape[1]

# Initialize variables
latitude = selected_sensor['Latitude'].values
longitude = selected_sensor['Longitude'].values
depth = selected_sensor['Depth'].values
range_values = selected_sensor['Range'].values

# Precompute distances and range matrix
dist_matrix = precompute_distances(selected_sensor, selected_sensor)
range_matrix = np.minimum(range_values[:, None], range_values[None, :])

# Start placing sensors
sensors = []
current_coverage = 0
remaining_indices = np.arange(len(selected_sensor))

# Create a priority queue to efficiently find the sensor with maximum coverage increase
pq = []

while current_coverage < 1:
    max_coverage_increase = 0
    best_sensor_index = None
    
    # Evaluate potential sensors and push them into the priority queue
    for i in remaining_indices:
        candidate_sensors = sensors + [i]
        new_coverage = calculate_coverage(candidate_sensors, dist_matrix, range_matrix)
        
        coverage_increase = new_coverage - current_coverage
        if coverage_increase > max_coverage_increase:
            max_coverage_increase = coverage_increase
            best_sensor_index = i
    
    if best_sensor_index is None:
        break
    
    sensors.append(best_sensor_index)
    current_coverage += max_coverage_increase
    print("Coverage:", current_coverage)
    
    if current_coverage >= 1:
        break
    
    # Remove the covered points from remaining_indices
    covered_by_best_sensor = dist_matrix[best_sensor_index, :] <= range_values[best_sensor_index]
    remaining_indices = remaining_indices[~covered_by_best_sensor[remaining_indices]]

# Convert sensor list to dataframe
sensors_df = selected_sensor.iloc[sensors].reset_index(drop=True)

# Output the new dataframe with sensor placements
print(sensors_df)


Coverage: 0.027194066749072928
Coverage: 0.054388133498145856
Coverage: 0.07787391841779975
Coverage: 0.10135970333745364
Coverage: 0.12360939431396786
Coverage: 0.14585908529048208
Coverage: 0.1668726823238566
Coverage: 0.18788627935723115
Coverage: 0.207663782447466
Coverage: 0.22744128553770088
Coverage: 0.24721878862793573
Coverage: 0.2669962917181706
Coverage: 0.2867737948084054
Coverage: 0.3053152039555006
Coverage: 0.3238566131025958
Coverage: 0.34239802224969096
Coverage: 0.36093943139678614
Coverage: 0.3794808405438813
Coverage: 0.3967861557478368
Coverage: 0.41409147095179233
Coverage: 0.43016069221260816
Coverage: 0.446229913473424
Coverage: 0.4622991347342398
Coverage: 0.47713226205191595
Coverage: 0.4919653893695921
Coverage: 0.5067985166872683
Coverage: 0.5216316440049443
Coverage: 0.5364647713226205
Coverage: 0.5512978986402967
Coverage: 0.5648949320148331
Coverage: 0.5784919653893696
Coverage: 0.5920889987639061
Coverage: 0.6044499381953028
Coverage: 0.6168108776266996


In [24]:
fig3 = go.Figure()
fig3.add_trace(go.Scatter3d(
    x=sensors_df['Latitude'], y=sensors_df['Longitude'], z=sensors_df['Depth'],
    mode='markers',
    marker=dict(
        size=4,
        color='blue',  # You can choose any color you prefer
        symbol='circle'
    ),
    name='New Point'
))
fig3.update_layout(
    title='3D Surface Map with Selected Sensor Locations',
    scene=dict(
        xaxis_title='Latitude',
        yaxis_title='Longitude',
        zaxis_title='Depth',
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)